# Предсказание стоимости жилья

**Задача проекта** — предсказать медианную стоимость дома в жилом массиве в Калифорнии на основе данных за 1990 год. Необходимо обучить модель линейной регрессии и сделать предсказания на тестовой выборке. В качестве метрик оценки будут использованы метрики *RMSE*, *MAE* и *R2*.

## Подготовка данных

In [14]:
# импорт библиотек и методов
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import when

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import RegressionMetrics

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

spark = SparkSession.builder \
        .master("local") \
        .appName("Training model") \
        .getOrCreate()

In [15]:
# чтение файла и вывод первых пяти строк 
df = spark.read.load('/datasets/housing.csv', format='csv', sep=',', inferSchema=True, header='true')
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [16]:
# просмотр типов данных
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [17]:
# просмотр количества пропусков 
df.select(*(F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns)).show(vertical=True)

-RECORD 0-----------------
 longitude          | 0   
 latitude           | 0   
 housing_median_age | 0   
 total_rooms        | 0   
 total_bedrooms     | 207 
 population         | 0   
 households         | 0   
 median_income      | 0   
 median_house_value | 0   
 ocean_proximity    | 0   



In [18]:
df.limit(5).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [19]:
# получение сводки числовых значений данных в датафрейме
df.describe().show(vertical=True)

-RECORD 0---------------------------------
 summary            | count               
 longitude          | 20640               
 latitude           | 20640               
 housing_median_age | 20640               
 total_rooms        | 20640               
 total_bedrooms     | 20433               
 population         | 20640               
 households         | 20640               
 median_income      | 20640               
 median_house_value | 20640               
 ocean_proximity    | 20640               
-RECORD 1---------------------------------
 summary            | mean                
 longitude          | -119.56970445736148 
 latitude           | 35.6318614341087    
 housing_median_age | 28.639486434108527  
 total_rooms        | 2635.7630813953488  
 total_bedrooms     | 537.8705525375618   
 population         | 1425.4767441860465  
 households         | 499.5396802325581   
 median_income      | 3.8706710029070246  
 median_house_value | 206855.81690891474  
 ocean_prox

**Исходные данные:**

В колонках датасета `housing` содержатся следующие данные:

* *longitude* — широта;
* *latitude* — долгота;
* *housing_median_age* — медианный возраст жителей жилого массива;
* *total_rooms* — общее количество комнат в домах жилого массива;
* *total_bedrooms* — общее количество спален в домах жилого массива;
* *population* — количество человек, которые проживают в жилом массиве;
* *households* — количество домовладений в жилом массиве;
* *median_income* — медианный доход жителей жилого массива;
* *median_house_value* — медианная стоимость дома в жилом массиве;
* *ocean_proximity* — близость к океану.

Проанализируем полученную информацию. Датасет содержит 10 колонок с типами данных:
* `double`
* `string`

Данные представлены числовыми и категориальными признаками. В колонке `total_bedrooms` содержатся пропуски. В названиях колонок использован стиль наименования переменных *lower_case_with_underscores*. В наименованиях отсутствуют нарушения стиля. Аномальных значений в сводке числовых данных не обнаружено.

Необходимо обработать пропущенные значения в столбце `total_bedrooms`. Заполним их с помощью медианного коэффициента соотношения количества спален в жилом массиве к общему количеству комнат в жилом массиве.

In [20]:
# создание колонки со вспомогательным коеффициентом
df_coef = df.withColumn('coef', F.column('total_bedrooms')/F.column('total_rooms'))
df_coef.show(5, vertical=True)

-RECORD 0---------------------------------
 longitude          | -122.23             
 latitude           | 37.88               
 housing_median_age | 41.0                
 total_rooms        | 880.0               
 total_bedrooms     | 129.0               
 population         | 322.0               
 households         | 126.0               
 median_income      | 8.3252              
 median_house_value | 452600.0            
 ocean_proximity    | NEAR BAY            
 coef               | 0.14659090909090908 
-RECORD 1---------------------------------
 longitude          | -122.22             
 latitude           | 37.86               
 housing_median_age | 21.0                
 total_rooms        | 7099.0              
 total_bedrooms     | 1106.0              
 population         | 2401.0              
 households         | 1138.0              
 median_income      | 8.3014              
 median_house_value | 358500.0            
 ocean_proximity    | NEAR BAY            
 coef      

In [21]:
# расчёт медианного коэффициента
coef_list =  df_coef.select('coef').collect()
coef_array = [row.coef for row in coef_list]
coef_array = [value for value in coef_array if value not in [None, np.nan]]
median_coef = round(np.median(coef_array), 3)
print(median_coef)

0.203


In [22]:
# расчёт нового столбца с количеством спален с помощью медианного коэффициента
df_housing = df_coef.withColumn('total_bedrooms_', 
                                    when(F.col('total_bedrooms').isNull(), F.col('total_rooms') * 0.203) \
                                        .otherwise(F.col('total_bedrooms')))

In [23]:
# проверка на наличие пропусков
df_housing.select(*(F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df_housing.columns)).show(vertical=True)

-RECORD 0-----------------
 longitude          | 0   
 latitude           | 0   
 housing_median_age | 0   
 total_rooms        | 0   
 total_bedrooms     | 207 
 population         | 0   
 households         | 0   
 median_income      | 0   
 median_house_value | 0   
 ocean_proximity    | 0   
 coef               | 207 
 total_bedrooms_    | 0   



В новой колонке `total_bedrooms_` пропусков не имеется. Колонки `coef`, `total_bedrooms` удалим, т.к. они больше не нужны.

In [24]:
# удаление столбцов
df_housing = df_housing.drop('total_bedrooms', 'coef')

In [25]:
# просмотр первых пяти строк
df_housing.show(5, vertical=True)

-RECORD 0----------------------
 longitude          | -122.23  
 latitude           | 37.88    
 housing_median_age | 41.0     
 total_rooms        | 880.0    
 population         | 322.0    
 households         | 126.0    
 median_income      | 8.3252   
 median_house_value | 452600.0 
 ocean_proximity    | NEAR BAY 
 total_bedrooms_    | 129.0    
-RECORD 1----------------------
 longitude          | -122.22  
 latitude           | 37.86    
 housing_median_age | 21.0     
 total_rooms        | 7099.0   
 population         | 2401.0   
 households         | 1138.0   
 median_income      | 8.3014   
 median_house_value | 358500.0 
 ocean_proximity    | NEAR BAY 
 total_bedrooms_    | 1106.0   
-RECORD 2----------------------
 longitude          | -122.24  
 latitude           | 37.85    
 housing_median_age | 52.0     
 total_rooms        | 1467.0   
 population         | 496.0    
 households         | 177.0    
 median_income      | 7.2574   
 median_house_value | 352100.0 
 ocean_p

Далее переходим к подготовке признаков датасета для последующей их передачи модели машинного обучения. В отдельные списки выделим категориальные, количественный признаки и целевой признак.

In [26]:
# признаки
categorical_cols = 'ocean_proximity'
numerical_cols  = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'population',
                   'households','median_income', 'total_bedrooms_']
# целевой признак
target = 'median_house_value' 

Трансформируем категориальные признаки с помощью трансформера *StringIndexer*, а затем дополнительно применим трансфомер *OneHotEncoder*.

In [27]:
# трансформация категориального признака с помощью StringIndexer
indexer = StringIndexer(inputCols=['ocean_proximity'], 
                        outputCols=['ocean_proximity_idx']) 
df_housing = indexer.fit(df_housing).transform(df_housing)

cols = ['ocean_proximity', 'ocean_proximity_idx']
df_housing.select(cols).show(3) 

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
+---------------+-------------------+
only showing top 3 rows



In [28]:
# трансформация категориального признака с помощью OneHotEncoder
encoder = OneHotEncoder(inputCols=['ocean_proximity_idx'],
                        outputCols=['ocean_proximity_ohe'])
df_housing = encoder.fit(df_housing).transform(df_housing)

cols = ['ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe']
df_housing.select(cols).show(3) 

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



Далее представим признаки в виде векторов для последующей передачи модели машинного обучения.

In [29]:
# векторизация категориального признака
categorical_assembler = VectorAssembler(inputCols=['ocean_proximity_ohe'], outputCol='categorical_features')
df_housing = categorical_assembler.transform(df_housing) 

In [30]:
# векторизация численных признаков
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol='numerical_features')
df_housing = numerical_assembler.transform(df_housing) 

Стандартизируем численные признаки так, чтобы выбросы не искажали общей картины.Будет использовать трансформер *StandardScaler*.

In [31]:
# стандартизация
standardScaler = StandardScaler(inputCol='numerical_features', outputCol='numerical_features_scaled')
df_housing = standardScaler.fit(df_housing).transform(df_housing) 

In [32]:
# вывод столбцов и их типов данных
df_housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- total_bedrooms_: double (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_ohe: vector (nullable = true)
 |-- categorical_features: vector (nullable = true)
 |-- numerical_features: vector (nullable = true)
 |-- numerical_features_scaled: vector (nullable = true)



Опробуем два набора даннных: один будет содержать только количественные признаки, другой — и количественные, и категориальный. На этих наборах данных обучим модель и сравним полученные результаты.

In [33]:
# создание двух наборов данных
all_features = ['categorical_features', 'numerical_features_scaled']

# датасет с категориальным признаком
final_assembler_cat = VectorAssembler(inputCols=all_features, outputCol='features') 
df_with_categorical = final_assembler_cat.transform(df_housing)

# датасет без категориального признака
df_without_categorical = df_housing.select('numerical_features_scaled', 'median_house_value')
df_without_categorical = df_without_categorical.selectExpr('numerical_features_scaled as features', 'median_house_value as median_house_value')

In [34]:
# просмотр первых пяти строк датасета
df_with_categorical.show(5, vertical=True)

-RECORD 0-----------------------------------------
 longitude                 | -122.23              
 latitude                  | 37.88                
 housing_median_age        | 41.0                 
 total_rooms               | 880.0                
 population                | 322.0                
 households                | 126.0                
 median_income             | 8.3252               
 median_house_value        | 452600.0             
 ocean_proximity           | NEAR BAY             
 total_bedrooms_           | 129.0                
 ocean_proximity_idx       | 3.0                  
 ocean_proximity_ohe       | (4,[3],[1.0])        
 categorical_features      | (4,[3],[1.0])        
 numerical_features        | [-122.23,37.88,41... 
 numerical_features_scaled | [-61.007269596069... 
 features                  | [0.0,0.0,0.0,1.0,... 
-RECORD 1-----------------------------------------
 longitude                 | -122.22              
 latitude                  | 37

In [35]:
# просмотр первых пяти строк датасета
df_without_categorical.show(5, vertical=True)

-RECORD 0----------------------------------
 features           | [-61.007269596069... 
 median_house_value | 452600.0             
-RECORD 1----------------------------------
 features           | [-61.002278409814... 
 median_house_value | 358500.0             
-RECORD 2----------------------------------
 features           | [-61.012260782324... 
 median_house_value | 352100.0             
-RECORD 3----------------------------------
 features           | [-61.017251968579... 
 median_house_value | 341300.0             
-RECORD 4----------------------------------
 features           | [-61.017251968579... 
 median_house_value | 342200.0             
only showing top 5 rows



**Вывод:**

Данные предобработаны и подготовлены к обучению.

## Обучение моделей

Для обучения будем использовать модель машинного обучения *LinearRegression()*.

Разделим наборы данных на обучающие и тестовые выборки.

In [36]:
# разделение датасетов на выборки
train_data_cat, test_data_cat = df_with_categorical.randomSplit([.8,.2], seed=12345)
train_data, test_data = df_without_categorical.randomSplit([.8,.2], seed=12345)

Инициализируем модель линейной регрессии, обучим её на двух наборах данных и сравним получившиеся метрики качества: *MAE*, *RMSE*, *R2*. Метрики качества выведем с помощью метода *summary*.

In [37]:
# инициализация модели линейной регрессии
lr = LinearRegression(labelCol=target, featuresCol='features').setRegParam(0.3)

In [38]:
# обучение модели и получение предсказаний для набора с категориальным и количественными признаками
model_cat = lr.fit(train_data_cat)
print('Метрики качества для модели с количественными признаками и категориальным')
print(f'MAE = {round(model_cat.summary.meanAbsoluteError, 2)}\nRMSE = {round(model_cat.summary.rootMeanSquaredError, 2)}\nR2 = {round(model_cat.summary.r2, 3)}')

22/06/04 21:21:59 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/06/04 21:21:59 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/06/04 21:21:59 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/06/04 21:21:59 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


Метрики качества для модели с количественными признаками и категориальным
MAE = 50014.58
RMSE = 68870.31
R2 = 0.643


In [39]:
# обучение модели и получение предсказаний для набора с количественными признаками
model = lr.fit(train_data)
print('Метрики качества для модели с количественными признаками и категориальным')
print(f'MAE = {round(model.summary.meanAbsoluteError, 2)}\nRMSE = {round(model.summary.rootMeanSquaredError, 2)}\nR2 = {round(model.summary.r2, 3)}')

Метрики качества для модели с количественными признаками и категориальным
MAE = 51060.59
RMSE = 69751.47
R2 = 0.634


Модель, обученная на данных с категориальным и количественными признаки, показала себя лучше. Остановим свой выбор на этой модели и оценим её качество на тестовой выборке.

Выведем предсказания выбранной модели на тестовой выборке.

In [26]:
# получение предсказаний для набора с категориальным и количественными признаками
predictions_cat = model_cat.transform(test_data_cat)
predictedLabes_cat = predictions_cat.select('median_house_value', 'prediction')
predictedLabes_cat.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          106700.0|215298.48617040692|
|          128900.0|  206450.312654607|
|          116100.0|231953.21266054967|
|           70500.0|161473.50755597558|
|           85600.0|186545.82507112948|
|           75500.0| 136330.3722085934|
|           79600.0|159894.33202837314|
|           92800.0|207100.82789575215|
|           97300.0|167092.13752041198|
|           82100.0|157800.22956366977|
|          126900.0|156428.79883326683|
|          119400.0|170070.58750063134|
|           71300.0|  169287.816466094|
|           75600.0| 148284.2972684563|
|           98800.0|165828.17444015061|
|           92600.0| 150725.4490463403|
|          152700.0|140671.83437411208|
|          150000.0|157081.52844457794|
|           74000.0|160147.45563937305|
|           82400.0| 163682.7567857164|
+------------------+------------------+
only showing top 20 rows



С помощью трансформера *RegressionEvaluator()* посмотриv на метрики *MAE*, *RMSE*, *R2*.

In [27]:
# вывод метрик качества тестовой выборки
metric_value = []
metrics = ['mae', 'rmse', 'r2']
evaluator = RegressionEvaluator()
evaluator.setLabelCol('median_house_value')

for metric in metrics:
    metric_value.append(round(evaluator.evaluate(predictedLabes_cat, {evaluator.metricName: metric}), 2))

print('Метрики качества для модели с количественными признаками и категориальным')
print(f'MAE = {metric_value[0]}\nRMSE = {metric_value[1]}\nR2 = {metric_value[2]}')             

Метрики качества для модели с количественными признаками и категориальным
MAE = 49051.03
RMSE = 67716.81
R2 = 0.66


In [28]:
# закрываем pySpark сессию
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f5d879c9220>>

**Вывод:**

Модели обучены, отобрана лучшая модель, метрики качества для оценки получены.

# Анализ результатов

В целом алгоритм линейной регрессии *LinearRegression()* показал себя не очень хорошо. Об этом можно судить по метрике *R2*, которая показывает, какое количество дисперсий (данных) покрывает модель. Значение метрики *R2* должно стремиться к 1.0, в данном примере мы получили значение метрики в 0.66. 

Алгоритм линейной регрессии ведет себя лучше на наборе данных с количественными и категориальными признаками вместе. Метрики MAE, RMSE, R2 у такого алгоритма выше.